# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [68]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import re

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [69]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('disaster_messages', engine)

df = df[:200]

X = df['message']
Y = df.iloc[:, 4:]

In [70]:
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [72]:
Y.head(3)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      200 non-null    int64  
 1   message                 200 non-null    object 
 2   original                200 non-null    object 
 3   genre                   200 non-null    object 
 4   related                 200 non-null    float64
 5   request                 200 non-null    float64
 6   offer                   200 non-null    float64
 7   aid_related             200 non-null    float64
 8   medical_help            200 non-null    float64
 9   medical_products        200 non-null    float64
 10  search_and_rescue       200 non-null    float64
 11  security                200 non-null    float64
 12  military                200 non-null    float64
 13  child_alone             200 non-null    float64
 14  water                   200 non-null    fl

### 2. Write a tokenization function to process your text data

In [74]:
def tokenize(text):
    '''Function that takes in a text splits with white spaces and creates list of words'''
    text = text.lower() 
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    word_list = nltk.tokenize.word_tokenize(text)
    word_list = [n for n in word_list if n not in stopwords.words("english")]
    
    lemmed_list = [WordNetLemmatizer().lemmatize(n, pos='v') for n in word_list]
    
    return lemmed_list

In [75]:
test_text = 'Hello world. This is just a test'
tokenize(test_text)

['hello', 'world', 'test']

Works!

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [76]:
# Setting up the machine learning model
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
        ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [77]:
# Step one: Split into train- and test data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.33)

In [78]:
# Our model to train is "pipeline"
model = pipeline.fit(X_train, y_train)

In [79]:
pred_train = model.predict(X_train)

In [80]:
pred_test = model.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [81]:
# print predictions classification_report()
for index, column in enumerate(y_test.columns):
    print('Category: {}'.format(column))
    print(classification_report(y_test[column], pred_test[:,index]))

Category: related
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        12
         1.0       0.82      1.00      0.90        54

    accuracy                           0.82        66
   macro avg       0.41      0.50      0.45        66
weighted avg       0.67      0.82      0.74        66

Category: request
              precision    recall  f1-score   support

         0.0       0.83      0.16      0.26        32
         1.0       0.55      0.97      0.70        34

    accuracy                           0.58        66
   macro avg       0.69      0.56      0.48        66
weighted avg       0.69      0.58      0.49        66

Category: offer
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99        65
         1.0       0.00      0.00      0.00         1

    accuracy                           0.98        66
   macro avg       0.49      0.50      0.50        66
weighted avg       0.9

In [82]:
# print predictions classification_report()
for index, column in enumerate(y_train.columns):
    print('Category: {}'.format(column))
    print(classification_report(y_train[column], pred_train[:,index]))

Category: related
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        17
         1.0       1.00      1.00      1.00       116
         2.0       1.00      1.00      1.00         1

    accuracy                           1.00       134
   macro avg       1.00      1.00      1.00       134
weighted avg       1.00      1.00      1.00       134

Category: request
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        49
         1.0       1.00      1.00      1.00        85

    accuracy                           1.00       134
   macro avg       1.00      1.00      1.00       134
weighted avg       1.00      1.00      1.00       134

Category: offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       134

    accuracy                           1.00       134
   macro avg       1.00      1.00      1.00       134
weighted avg       1.0

### 6. Improve your model
Use grid search to find better parameters.

In [83]:
# What are the parameters of the pipeline object?
params = pipeline.get_params().keys()

In [84]:
for param in params:
    print(param + ': ', pipeline.get_params()[param], '\n')

memory:  None 

steps:  [('vect', CountVectorizer(tokenizer=<function tokenize at 0x00000260B450A040>)), ('tfidf', TfidfTransformer()), ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))] 

verbose:  False 

vect:  CountVectorizer(tokenizer=<function tokenize at 0x00000260B450A040>) 

tfidf:  TfidfTransformer() 

clf:  MultiOutputClassifier(estimator=RandomForestClassifier()) 

vect__analyzer:  word 

vect__binary:  False 

vect__decode_error:  strict 

vect__dtype:  <class 'numpy.int64'> 

vect__encoding:  utf-8 

vect__input:  content 

vect__lowercase:  True 

vect__max_df:  1.0 

vect__max_features:  None 

vect__min_df:  1 

vect__ngram_range:  (1, 1) 

vect__preprocessor:  None 

vect__stop_words:  None 

vect__strip_accents:  None 

vect__token_pattern:  (?u)\b\w\w+\b 

vect__tokenizer:  <function tokenize at 0x00000260B450A040> 

vect__vocabulary:  None 

tfidf__norm:  l2 

tfidf__smooth_idf:  True 

tfidf__sublinear_tf:  False 

tfidf__use_idf:  True 

clf__est

#### Selecting some example parameters for the text pipeline:
- text_pipeline__vect__max_df:  1.0
- text_pipeline__vect__min_df:  1
- text_pipeline__vect__max_features:  None
- text_pipeline__vect__stop_words:  None
- text_pipeline__tfidf__use_idf:  True

#### Selecting some example parameters for the classifier pipeline:
- clf__estimator__max_depth:  None
- clf__estimator__min_samples_leaf:  1
- clf__estimator__n_estimators:  10

In [85]:
# Trying new parameters for grid search
parameters = {
    'text_pipeline__vect__max_df': (0.25, 0.5, 0.75, 1.0),
    #'text_pipeline__vect__min_df': (1, 1.25, 1.5, 1.75),
    #'text_pipeline__vect__max_features': (None, 1000, 500),
    #'text_pipeline__vect__stop_words': (None, 1000, 200),
    'text_pipeline__tfidf__use_idf': (True, False),
    #'clf__estimator__max_depth': (None, 1000),
    #'clf__estimator__min_samples_leaf': (1, 5, 10),
    'clf__estimator__n_estimators': (10, 50, 100)
    }

# Apply parameters to Gridsearch
cv = GridSearchCV(pipeline, param_grid=parameters)

In [86]:
cv.fit(X_train, y_train)

ValueError: Invalid parameter text_pipeline for estimator Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000260B450A040>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))]). Check the list of available parameters with `estimator.get_params().keys()`.

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.